In [1]:
!pip install pydantic==2.7.0
!pip install langchain
!pip install langchain-openai
!pip install tiktoken

In [2]:
import os

# # Set OPENAI API Key

os.environ["OPENAI_API_KEY"] = "your openai key"

# OR (load from .env file)

# from dotenv import load_dotenv
# make sure you have python-dotenv installed
# load_dotenv("./.env")

When we want to convert openai functions to pydantic objects (for a more structured approach for defining tools for LLMs) we can leverage the following ideas:

What is the goal here?
To go from openai functions to a Pydantic object that is an easier way to interact with structured information.

In [1]:
from pydantic import BaseModel, Field
from langchain_core.utils.function_calling import convert_to_openai_function
# this function will convert an openai function to a pydantic object


# this is a Pydantic object that inherits from the BaseModel class from pydantic and contains the fields that will be used in the openai function
class WeatherSearch(BaseModel):
    """Call this function to get the weather of a specific airport."""
    airport_code: str = Field(description="The airport code of the airport you want to know the weather of.")
    

openai_function_weather_search = convert_to_openai_function(WeatherSearch)

SO the airport_code is something that the LLM will figure out on how to input to the function, therefore making the call with the correct input.

In [2]:
from langchain_openai import ChatOpenAI

llm_chat = ChatOpenAI()

llm_chat.invoke("What's the weather like in Lisbon today?",functions=[openai_function_weather_search])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"LIS"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 60, 'total_tokens': 77}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-b58e4db3-4026-4dfa-b980-6b4525e279e2-0', usage_metadata={'input_tokens': 60, 'output_tokens': 17, 'total_tokens': 77})

We can see from below that the LLM figured out that the airport_code for Lisbon is `LIS`.

You can also bind the function to the model to avoid having to pass the `functions` keyword argument everytime. 

In [3]:
model_with_function = llm_chat.bind(functions=[openai_function_weather_search])
model_with_function.invoke("What's the weather like in Lisbon today?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"LIS"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 60, 'total_tokens': 77}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-2e4ed22d-4d7c-4b08-bf5d-d8915611ce56-0', usage_metadata={'input_tokens': 60, 'output_tokens': 17, 'total_tokens': 77})

YOu can also force the model to use the function by binding it with it and adding the `function_call` keyword argument to a dictionary containing the key `name` and the name of that function `WeatherSearch`:

In [4]:
model_with_forced_function = llm_chat.bind(functions=[openai_function_weather_search],function_call={"name": "WeatherSearch"})

Now, let's connect to the stuff we already know.

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that tells the weather"),
    ("user", "{input}")
])

llm_chat = ChatOpenAI(temperature=0)

llm_chat_with_forced_function = llm_chat.bind(functions=[openai_function_weather_search], function_call={"name": "WeatherSearch"})

chain = prompt | llm_chat_with_forced_function

chain.invoke({"input": "What's the weather like in Lisbon today?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"LIS"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 80, 'total_tokens': 87}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9c4a1b57-c55b-4be4-a74f-17e544e76e2a-0', usage_metadata={'input_tokens': 80, 'output_tokens': 7, 'total_tokens': 87})

For multiple functions:

In [6]:
from typing import List

class SearchExamples(BaseModel):
    """Call this function to search for examples of a concept."""
    concept: str = Field(description="The concept you want to search for.")
    
class AnalogySearch(BaseModel):
    """Call this function to search for analogies of a concept."""
    concept: str = Field(description="The concept you want to search for.")


openai_function_analogy_search = convert_to_openai_function(AnalogySearch)
openai_function_example_search = convert_to_openai_function(SearchExamples) 

In [7]:
functions = [
    openai_function_analogy_search,
    openai_function_example_search
]

In [8]:
llm_chat_with_functions = llm_chat.bind(functions=functions)

In [9]:
# Here the model calls the example function
llm_chat_with_functions.invoke("List examples for what and SDK is.")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"concept":"SDK"}', 'name': 'SearchExamples'}}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 85, 'total_tokens': 99}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-24cb4d9c-8a41-4887-bbfe-0d46ea67396f-0', usage_metadata={'input_tokens': 85, 'output_tokens': 14, 'total_tokens': 99})

In [10]:
llm_chat_with_functions.invoke("List analogies for what and SDK is.")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"concept":"SDK"}', 'name': 'AnalogySearch'}}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 86, 'total_tokens': 101}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-b8506fb4-8e35-4a39-a61e-62ea90698376-0', usage_metadata={'input_tokens': 86, 'output_tokens': 15, 'total_tokens': 101})